In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tensorflow.keras import layers
from tensorflow import keras
import tensorflow as tf
from sklearn.model_selection import train_test_split
from ast import literal_eval
import matplotlib.pyplot as plt

In [3]:
transactions_train = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/transactions_train.csv')

transactions_train['t_dat_datetime']  = pd.to_datetime(transactions_train['t_dat'])

transactions_train['t_month'] = transactions_train['t_dat_datetime'].dt.month

transactions_train['t_year']  = transactions_train['t_dat_datetime'].dt.year

transactions_train['customer_max_tran']  = transactions_train.\
groupby(['customer_id'])['t_dat_datetime'].transform('max')

In [4]:
cust_data = transactions_train[(transactions_train['customer_max_tran'].dt.year==2020)&
                                     (transactions_train['customer_max_tran'].dt.month == 9)]

cust_data['article_id'] = 'a'+cust_data['article_id'].astype('str')

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [5]:
cust_data_2020_9 = cust_data.groupby(['customer_id','article_id']).agg(tran_count = ('t_dat_datetime','nunique')).\
assign(Rank = lambda x : x.groupby('customer_id')['tran_count'].rank(method='first', ascending=False)
      ).\
reset_index().\
query('Rank <= 12').\
groupby('customer_id').agg(prods = ('article_id','unique')).reset_index().\
assign(prods = lambda x:  list(x['prods']))


# cust_data_2020_9["prods"] = cust_data_2020_9["prods"].apply( lambda x : list(x) ) 

In [6]:
cust_data_2020_9.head()

,customer_id,prods
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,"[a176209023, a568601006, a568601043, a60764200..."
1,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,"[a351484002, a541518023, a578020002, a66371300..."
2,0000757967448a6cb83efb3ea7a3fb9d418ac7adf2379d...,"[a448509014, a715624008, a719530003, a72692500..."
3,000172a9c322560c849754ffbdfdb2180d408aa7176b94...,"[a181448103, a309864002, a309864013, a38060903..."
4,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,"[a297791001, a304692003, a536933002, a66853700..."


In [7]:
article_df =  pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/articles.csv')
customer_df = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/customers.csv')

article_df['article_id'] = 'a'+article_df['article_id'].astype('str')

In [8]:
cust_data = pd.merge(pd.merge(cust_data ,
         article_df[['article_id','product_group_name']],
         left_on = 'article_id',
         right_on = 'article_id'
        ),
         customer_df[['customer_id','age']],
         left_on = 'customer_id',
         right_on = 'customer_id'
        )

In [9]:
cust_features = cust_data.groupby(['customer_id','age']).agg( tran_count = ('t_dat','nunique'),
                                       article_count = ('article_id','nunique'),
                                       spend = ('price','sum')
                                      ).reset_index()

In [10]:
cust_features_1 = pd.pivot_table(cust_data[['customer_id','product_group_name','t_dat']],
         index = 'customer_id',
         columns = 'product_group_name',
         values = 't_dat',
         aggfunc = 'nunique'
        ).fillna(0).reset_index()

In [11]:
cust_features = pd.merge(cust_features, 
                         cust_features_1,
                         left_on = 'customer_id',
                         right_on = 'customer_id',
                        )

In [12]:
cust_features.head()

,customer_id,age,tran_count,article_count,spend,Accessories,Bags,Cosmetic,Fun,Furniture,...,Interior textile,Items,Nightwear,Shoes,Socks & Tights,Stationery,Swimwear,Underwear,Underwear/nightwear,Unknown
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,49.0,10,19,0.648983,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,24.0,7,14,0.704780,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0
2,0000757967448a6cb83efb3ea7a3fb9d418ac7adf2379d...,20.0,3,6,0.166000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,000172a9c322560c849754ffbdfdb2180d408aa7176b94...,45.0,7,23,0.565203,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0
4,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,44.0,144,555,25.466898,23.0,0.0,0.0,0.0,0.0,...,0.0,0.0,5.0,14.0,7.0,0.0,10.0,14.0,0.0,1.0


In [13]:
# cust_data_2020_9["prods"] = cust_data_2020_9["prods"].apply( lambda x : list(x) ) 
# cust_data_2020_9["prods"].values[:5]

In [13]:
model_data = pd.merge(cust_data_2020_9,
                      cust_features,
                      left_on = 'customer_id',
                      right_on = 'customer_id'
                     )

In [14]:
import gc

del cust_data_2020_9
gc.collect()

del cust_features
gc.collect()

0

In [15]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()

In [16]:
genre_mlb = mlb.fit_transform(model_data['prods'])
genre_mlb

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [24]:
genre = mlb.inverse_transform(genre_mlb[0:1])
genre

[('a176209023',
  'a568601006',
  'a568601043',
  'a607642008',
  'a625548001',
  'a627759010',
  'a656719005',
  'a694736004',
  'a697138006',
  'a745232001',
  'a785186005',
  'a785710001')]

In [ ]:
# model_data['label'] = genre_mlb

In [25]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.optimizers import SGD

In [26]:
model_data.iloc[0:1000,2:]

,age,tran_count,article_count,spend,Accessories,Bags,Cosmetic,Fun,Furniture,Garment Full body,...,Interior textile,Items,Nightwear,Shoes,Socks & Tights,Stationery,Swimwear,Underwear,Underwear/nightwear,Unknown
0,49.0,10,19,0.648983,1.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,24.0,7,14,0.704780,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0
2,20.0,3,6,0.166000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,45.0,7,23,0.565203,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0
4,44.0,144,555,25.466898,23.0,0.0,0.0,0.0,0.0,55.0,...,0.0,0.0,5.0,14.0,7.0,0.0,10.0,14.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,22.0,2,6,0.224644,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
996,26.0,1,2,0.016746,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
997,35.0,13,39,0.976576,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,2.0,0.0,6.0,0.0,0.0,6.0,0.0,0.0
998,22.0,8,12,0.382492,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.0,0.0,3.0,2.0,0.0,0.0


In [27]:
X_train = model_data.iloc[0:1000,2:]
y_train = genre_mlb[0:1000]
X_test = model_data.iloc[1000:1100,2:]
y_test = genre_mlb[1000:1100]

In [28]:
model = Sequential()
model.add(Dense(5000, activation='relu', input_dim=X_train.shape[1]))
model.add(Dropout(0.1))
model.add(Dense(600, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(y_train.shape[1], activation='sigmoid'))

2022-04-03 17:41:28.910891: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [29]:
X_train.columns

Index(['age', 'tran_count', 'article_count', 'spend', 'Accessories', 'Bags',
       'Cosmetic', 'Fun', 'Furniture', 'Garment Full body',
       'Garment Lower body', 'Garment Upper body', 'Garment and Shoe care',
       'Interior textile', 'Items', 'Nightwear', 'Shoes', 'Socks & Tights',
       'Stationery', 'Swimwear', 'Underwear', 'Underwear/nightwear',
       'Unknown'],
      dtype='object')

In [30]:
def macro_soft_f1(y, y_hat):
    """Compute the macro soft F1-score as a cost.
    Average (1 - soft-F1) across all labels.
    Use probability values instead of binary predictions.
    
    Args:
        y (int32 Tensor): targets array of shape (BATCH_SIZE, N_LABELS)
        y_hat (float32 Tensor): probability matrix of shape (BATCH_SIZE, N_LABELS)
        
    Returns:
        cost (scalar Tensor): value of the cost function for the batch
    """
    
    y = tf.cast(y, tf.float32)
    y_hat = tf.cast(y_hat, tf.float32)
    tp = tf.reduce_sum(y_hat * y, axis=0)
    fp = tf.reduce_sum(y_hat * (1 - y), axis=0)
    fn = tf.reduce_sum((1 - y_hat) * y, axis=0)
    soft_f1 = 2*tp / (2*tp + fn + fp + 1e-16)
    cost = 1 - soft_f1 # reduce 1 - soft-f1 in order to increase soft-f1
    macro_cost = tf.reduce_mean(cost) # average on all labels
    
    return macro_cost

In [31]:
def macro_f1(y, y_hat, thresh=0.5):
    """Compute the macro F1-score on a batch of observations (average F1 across labels)
    
    Args:
        y (int32 Tensor): labels array of shape (BATCH_SIZE, N_LABELS)
        y_hat (float32 Tensor): probability matrix from forward propagation of shape (BATCH_SIZE, N_LABELS)
        thresh: probability value above which we predict positive
        
    Returns:
        macro_f1 (scalar Tensor): value of macro F1 for the batch
    """
    y_pred = tf.cast(tf.greater(y_hat, thresh), tf.float32)
    tp = tf.cast(tf.math.count_nonzero(y_pred * y, axis=0), tf.float32)
    fp = tf.cast(tf.math.count_nonzero(y_pred * (1 - y), axis=0), tf.float32)
    fn = tf.cast(tf.math.count_nonzero((1 - y_pred) * y, axis=0), tf.float32)
    f1 = 2*tp / (2*tp + fn + fp + 1e-16)
    macro_f1 = tf.reduce_mean(f1)
    return macro_f1

In [ ]:
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss=macro_soft_f1,
  metrics=[macro_f1], optimizer=sgd)

model.fit(X_train, y_train, epochs=1, batch_size=10)

/opt/conda/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")
2022-04-03 17:41:38.092592: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


 61/100 [=================>............] - ETA: 5s - loss: 0.9998 - macro_f1: 1.7675e-04

In [55]:
preds = model.predict(X_test)
preds[preds>=0.5] = 1
preds[preds<0.5] = 0
# score = compare preds and y_test

In [56]:
len(preds[0])

74282

In [38]:
score = model.evaluate(X_test, y_test, batch_size=10)
score

10/10 [==============================] - 0s 33ms/step - loss: 0.6804


0.6804035305976868

In [57]:
mlb.inverse_transform(preds[0:1])

[('a108775044',
  'a108775051',
  'a111565003',
  'a111586001',
  'a111609001',
  'a114428030',
  'a118458004',
  'a118458029',
  'a120129001',
  'a120129014',
  'a120129018',
  'a123173001',
  'a126589006',
  'a126589007',
  'a126589010',
  'a126589012',
  'a145872037',
  'a145872051',
  'a146730001',
  'a150959011',
  'a150959013',
  'a153115019',
  'a153115021',
  'a156231001',
  'a156231002',
  'a156610010',
  'a160442010',
  'a162074062',
  'a174057022',
  'a174057027',
  'a174057029',
  'a174057030',
  'a174057032',
  'a174057033',
  'a174057037',
  'a174057038',
  'a174057040',
  'a176209033',
  'a176209044',
  'a179123040',
  'a179208008',
  'a181160009',
  'a181448022',
  'a181448102',
  'a181448104',
  'a181448109',
  'a184123020',
  'a184583014',
  'a186262007',
  'a186264014',
  'a186372011',
  'a187949026',
  'a187949028',
  'a187949031',
  'a188183008',
  'a188183009',
  'a188183014',
  'a188183015',
  'a188183020',
  'a189383001',
  'a189383014',
  'a189616015',
  'a1896

In [50]:
literal_eval(mlb.inverse_transform(preds[0:1]))

ValueError: malformed node or string: [('a108775044', 'a108775051', 'a111565003', 'a111586001', 'a111609001', 'a114428030', 'a118458004', 'a118458029', 'a120129001', 'a120129014', 'a120129018', 'a123173001', 'a126589006', 'a126589007', 'a126589010', 'a126589012', 'a145872037', 'a145872051', 'a146730001', 'a150959011', 'a150959013', 'a153115019', 'a153115021', 'a156231001', 'a156231002', 'a156610010', 'a160442010', 'a162074062', 'a174057022', 'a174057027', 'a174057029', 'a174057030', 'a174057032', 'a174057033', 'a174057037', 'a174057038', 'a174057040', 'a176209033', 'a176209044', 'a179123040', 'a179208008', 'a181160009', 'a181448022', 'a181448102', 'a181448104', 'a181448109', 'a184123020', 'a184583014', 'a186262007', 'a186264014', 'a186372011', 'a187949026', 'a187949028', 'a187949031', 'a188183008', 'a188183009', 'a188183014', 'a188183015', 'a188183020', 'a189383001', 'a189383014', 'a189616015', 'a189616038', 'a189634001', 'a189654001', 'a189654045', 'a189654047', 'a189691033', 'a189691

In [ ]:
submission = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/sample_submission.csv')

submission['prediction'] = top_12_products.article_id.sum()

submission.to_csv('submission.csv', index = False)